In [2]:

import pandas as pd
import tensorflow as tf

/mnt/d/pyTensor/tf217/tf217/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-16 12:38:43.576961: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1731760724.902383     849 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731760725.257703     849 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-16 12:38:48.562343: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical 

In [7]:
df = pd.read_csv('preprocessed_data.csv')
df.head()

/tmp/ipykernel_733/1469898370.py:1: DtypeWarning: Columns (4,6,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('preprocessed_data.csv')


,author,author_created_utc,author_fullname,id,num_comments,edited,over_18,retrieved_on,retrieved_utc,score,selftext,subreddit,subreddit_id,subreddit_subscribers,title,total_awards_received,upvote_ratio,word_count
0,KYSUAPE,24-06-21 14:55,t2_cwvv6035,olpcjt,11,FALSE,TRUE,26-06-22 1:32,26-06-22 1:32,2.0,Hello there everyone ! :) \n\nI would like to ...,Drugs,t5_2qh7l,884547.0,Black &amp; Brown Kratom,0.0,1.00,33.0
1,Pleasant_Bus1179,29-01-21 21:03,t2_9tr0fjns,olpfwg,20,FALSE,TRUE,26-06-22 1:32,26-06-22 1:32,1.0,I got 5g of weed 1g of coke 1 vodka and 2 stro...,Drugs,t5_2qh7l,884547.0,How to crossfade with booze and weed??,0.0,0.56,38.0
2,Dazzling-Disk-632,28-01-21 15:43,t2_a1gg2f3a,olpgi6,6,FALSE,TRUE,26-06-22 1:32,26-06-22 1:32,0.0,Has any body else here smoke so much crack you...,Drugs,t5_2qh7l,884547.0,Has any 1 Hylucinated on crack,0.0,0.38,32.0
3,spiritually-high,31-03-21 16:56,t2_9lbpxsa0,olpv4a,132,FALSE,TRUE,26-06-22 1:31,26-06-22 1:31,138.0,I got hella anxiety and I’m hella coked up is ...,Drugs,t5_2qh7l,884547.0,can cocaine give anxiety?,0.0,0.94,21.0
4,Frenchbullmese,04-05-21 11:16,t2_bxihe2ze,olqjjx,14,FALSE,TRUE,26-06-22 1:31,26-06-22 1:31,18.0,I haven’t been so rude as to say these things ...,Drugs,t5_2qh7l,884546.0,Help me lol. My mom has been talking to my cou...,0.0,0.96,532.0


In [8]:
# Adding sentiment column
df['sentiment'] = np.nan

In [9]:
df.columns

Index(['author', 'author_created_utc', 'author_fullname', 'id', 'num_comments',
       'edited', 'over_18', 'retrieved_on', 'retrieved_utc', 'score',
       'selftext', 'subreddit', 'subreddit_id', 'subreddit_subscribers',
       'title', 'total_awards_received', 'upvote_ratio', 'word_count',
       'sentiment'],
      dtype='object')

In [10]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [12]:
sentiment_analysis = pipeline("sentiment-analysis" , framework="tf")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
I0000 00:00:1731604046.283603     733 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2242 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [15]:
from transformers import pipeline
import pandas as pd
import tensorflow as tf

# Initialize the sentiment analysis pipeline with TensorFlow backend
sentiment_analysis = pipeline("sentiment-analysis", framework="tf")

# Load your dataset
df = pd.read_csv('/mnt/d/pyTensor/tf217/Reddit/preprocessed_data.csv')

# Define batch size
batch_size = 16  # You can adjust this based on your GPU memory

# Function to analyze sentiment in batches
def analyze_sentiment_batch(texts):
    results = sentiment_analysis(texts)
    sentiments = []
    for result in results:
        score = result['score']
        label = result['label']
        
        if label == 'POSITIVE' and score >= 0.6:
            sentiments.append('positive')
        elif label == 'NEGATIVE' and score >= 0.6:
            sentiments.append('negative')
        else:
            sentiments.append('neutral')
    
    return sentiments

# Process the selftext column in batches
texts = df['selftext'].fillna("").tolist()  # Replace non-string values with empty strings
sentiments = []

# Process data in batches and collect sentiment results
for i in range(0, len(texts), batch_size):
    batch = texts[i:i + batch_size]
    sentiments.extend(analyze_sentiment_batch(batch))

# Add results to DataFrame
df['sentiment'] = sentiments

# Save the DataFrame with the new sentiment column
df.to_csv('/mnt/d/pyTensor/tf217/Reddit/sentiment_data.csv', index=False)
print("Sentiment Analysis Completed")


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
/tmp/ipykernel_733/25970713.py:9: DtypeWarning: Columns (4,6,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/mnt/d/pyTensor/tf217/Reddit/preprocessed_data.csv')
Token indices sequence length is longer than the specified maximum sequence length for this model (676 > 512). Runnin

Sentiment Analysis Completed


In [17]:
df['sentiment'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [18]:
df['sentiment'].value_counts()

sentiment
negative    74139
positive    18414
neutral      1529
Name: count, dtype: int64

In [3]:
df2 = pd.read_csv('sentiment_data.csv')

/tmp/ipykernel_849/1418777235.py:1: DtypeWarning: Columns (4,6,9,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('sentiment_data.csv')


In [12]:
df2.isnull().sum()

author                    78
author_created_utc       102
author_fullname          119
id                       133
num_comments             139
edited                   143
over_18                  148
retrieved_on             149
retrieved_utc            155
score                    158
selftext                 158
subreddit                163
subreddit_id             165
subreddit_subscribers    167
title                    168
total_awards_received    170
upvote_ratio             170
word_count               171
sentiment                  0
dtype: int64

In [7]:
df2.shape[0]

94082

In [13]:
#Removing NAN values from the datset
df2.dropna(subset=['author', 'author_created_utc', 'author_fullname', 'id', 'num_comments', 'edited', 'over_18', 'retrieved_on', 'retrieved_utc', 'score', 'selftext', 'subreddit', 'subreddit_id', 'subreddit_subscribers', 'title', 'total_awards_received', 'upvote_ratio', 'word_count'], inplace=True)

In [14]:
df2.isnull().sum()

author                   0
author_created_utc       0
author_fullname          0
id                       0
num_comments             0
edited                   0
over_18                  0
retrieved_on             0
retrieved_utc            0
score                    0
selftext                 0
subreddit                0
subreddit_id             0
subreddit_subscribers    0
title                    0
total_awards_received    0
upvote_ratio             0
word_count               0
sentiment                0
dtype: int64

In [15]:
df2.shape[0]

93911

In [21]:
df2['author_created_utc'].head(10)

0    24-06-21 14:55
1    29-01-21 21:03
2    28-01-21 15:43
3    31-03-21 16:56
4    04-05-21 11:16
5     20-05-21 1:58
6    01-03-20 22:11
7    28-11-18 18:01
8    09-02-15 19:38
9    03-11-20 22:27
Name: author_created_utc, dtype: object

In [22]:
df2['author_created_utc'] = pd.to_datetime(df2['author_created_utc'], format='%d-%m-%y %H:%M', errors='coerce')
df2['retrieved_on'] = pd.to_datetime(df2['retrieved_on'], format='%d-%m-%y %H:%M', errors='coerce')
df2['retrieved_utc'] = pd.to_datetime(df2['retrieved_utc'], format='%d-%m-%y %H:%M', errors='coerce')

In [23]:
output_file = '/mnt/d/pyTensor/tf217/Reddit/updated_sentiment_data.csv'
df2.to_csv(output_file, index=False)